In [7]:
import logging
import os
import sys
import math
import argparse
sys.path.append('scripts')
sys.path.append('share')
sys.path.append('python')
from collections import namedtuple
import numpy as np
import pandas as pd
import h5py as h5
import matplotlib.pyplot as plt
import tensorflow
import tensorflow.keras as keras
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc

In [8]:
def doNumber(data_EC, data_Layer, data_number, data_true):

    data = {}

    outdata = np.zeros(
        (data_number.shape[0],),
        dtype=[
            ('Output_number', np.dtype(('f4', 3))),
            ('Output_number_true', 'i4')
        ]
    )

    IBL = np.logical_and(data_Layer == 0, data_EC == 0)
    Barrel = np.logical_and(data_Layer > 0, data_EC == 0)
    Endcap = data_EC != 0

    outdata['Output_number'] = data_number
    outdata['Output_number_true'] = data_true

    data['IBL'] = outdata[IBL]
    data['Barrel'] = outdata[Barrel]
    data['Endcap'] = outdata[Endcap]
    return data

In [9]:
def evaluate(data, classes):

    fpr = [[0] for i in range(len(data))]
    tpr = [[0] for i in range(len(data))]
    auc1 = [[0] for i in range(len(data))]
    
    pos, neg = classes

    for i, layer in enumerate(data):

        pos_sel = data[layer]['Output_number_true'] == pos
        neg_sel = data[layer]['Output_number_true'] == neg

        isel = np.where(
            np.logical_or(
                pos_sel,
                neg_sel,
            )
        )[0]

        fpr[i], tpr[i], _ = roc_curve(
            data[layer]['Output_number_true'][isel],
            data[layer]['Output_number'][isel][:, pos - 1],
            pos_label=pos
        )
        auc1[i] = auc(fpr[i], tpr[i])

    return fpr, tpr, auc1

In [10]:
def rocGraph(dataAll, dataLG, dataTotal, classes):

    pos, neg = classes
    All_fpr, All_tpr, All_auc = evaluate(dataAll, classes)
    LG_fpr, LG_tpr, LG_auc = evaluate(dataLG, classes)
    Total_fpr, Total_tpr, Total_auc = evaluate(dataTotal, classes)
        
    for i, layer in enumerate(dataAll):
        
        plt.plot(All_fpr[i], All_tpr[i], label=f'1 Model (AUC = {All_auc[i]:.2f}')
        plt.plot(LG_fpr[i], LG_tpr[i], label=f'LG Model (AUC = {LG_auc[i]:.2f}')
        plt.plot(Total_fpr[i], Total_tpr[i], label=f'3 Model (AUC = {Total_auc[i]:.2f}')
        
        rand_chance = np.linspace(0, 1, 100)
        plt.plot(rand_chance,rand_chance, ':', color='grey', label='Random (AUC = 0.5)')
        plt.semilogx()
        plt.ylabel(f"Pr(Estimated: {pos}-particle | True: {pos}-particle)")
        plt.xlabel(f"Pr(Estimated: {pos}-particle | True: {neg}-particle)")
        plt.xlim([0.001, 1.05])
        plt.ylim(0,1.5)
        plt.legend(loc='upper left')
        plt.grid(True)
        plt.figtext(0.25, 0.90,f'{layer} {pos} vs {neg}',fontweight='bold', wrap=True, horizontalalignment='right', fontsize=10)
        plt.savefig(f'output/{layer}_{pos}vs{neg}_ROC.png')
        plt.close()

In [11]:
def doRocs(dataAll, dataLG, dataTotal):
    rocGraph(dataAll, dataLG, dataTotal, (3, 2))
    rocGraph(dataAll, dataLG, dataTotal, (3, 1))
    rocGraph(dataAll, dataLG, dataTotal, (2, 3))
    rocGraph(dataAll, dataLG, dataTotal, (2, 1))
    rocGraph(dataAll, dataLG, dataTotal, (1, 2))
    rocGraph(dataAll, dataLG, dataTotal, (1, 3))

In [12]:
LGModel = "LG_apply"
alltrainedModel = "theano_kfold/theano_kfold1"
ECtrained = "endcap_flip_trained_apply"
Barreltrained = "barrel_trained_apply"
IBLtrained = "ibl_trained_apply"

with h5.File(f'output/{LGModel}.h5', 'r') as data:
    data_EC = data['NN_barrelEC'][()]
    data_Layer = data['NN_layer'][()]
    data_number = data['Output_number'][()]
    data_true = data['Output_number_true'][()]

dataLG = doNumber(data_EC, data_Layer, data_number, data_true)

with h5.File(f'output/{alltrainedModel}.h5', 'r') as data:
    data_EC = data['NN_barrelEC'][()]
    data_Layer = data['NN_layer'][()]
    data_number = data['Output_number'][()]
    data_true = data['Output_number_true'][()]

dataAll = doNumber(data_EC, data_Layer, data_number, data_true)

with h5.File(f'output/{ECtrained}.h5', 'r') as data:
    data_EC = data['NN_barrelEC'][()]
    data_Layer = data['NN_layer'][()]
    data_number = data['Output_number'][()]
    data_true = data['Output_number_true'][()]

dataEC = doNumber(data_EC, data_Layer, data_number, data_true)

with h5.File(f'output/{Barreltrained}.h5', 'r') as data:
    data_EC = data['NN_barrelEC'][()]
    data_Layer = data['NN_layer'][()]
    data_number = data['Output_number'][()]
    data_true = data['Output_number_true'][()]

dataBarrel = doNumber(data_EC, data_Layer, data_number, data_true)

with h5.File(f'output/{IBLtrained}.h5', 'r') as data:
    data_EC = data['NN_barrelEC'][()]
    data_Layer = data['NN_layer'][()]
    data_number = data['Output_number'][()]
    data_true = data['Output_number_true'][()]

dataIBL = doNumber(data_EC, data_Layer, data_number, data_true)

dataTotal = {'IBL':dataIBL['IBL'], 'Barrel':dataBarrel['Barrel'], 'Endcap':dataEC['Endcap']}

doRocs(dataAll, dataLG, dataTotal)